In [1]:
print("Hii Yash, you are a billionior by 2028 keep grinding!!")

Hii Yash, you are a billionior by 2028 keep grinding!!


In [4]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import schedule
import time

def connect_mt5():
    if not mt5.initialize():
        raise RuntimeError(" !!! Error >>> MT5 initialize() failed")
    print("Good! Connected to MT5")

def shutdown_mt5():
    mt5.shutdown()
    print("MT5 disconnected")

def get_fifteen_min_data(symbol="EURUSD", n=10):
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M15, 0, n)
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

def detect_engulfing(df):
    if len(df) < 2:
        return "neutral"
    prev = df.iloc[-2]
    curr = df.iloc[-1]

    if prev['close'] < prev['open'] and curr['close'] > curr['open']:
        if curr['close'] > prev['open'] and curr['open'] < prev['close']:
            return "bullish"
    if prev['close'] > prev['open'] and curr['close'] < curr['open']:
        if curr['open'] > prev['close'] and curr['close'] < prev['open']:
            return "bearish"
    return "neutral"

def place_trade(symbol, signal, df, lot=0.1):
    # signal = "bullish" # !!! using this line just to test the function and MT5 connection
    if signal not in ["bullish", "bearish"]:
        print("No trade signal (neutral) — skipping trade.")
        return

    tick = mt5.symbol_info_tick(symbol)
    price = tick.ask if signal == "bullish" else tick.bid
    order_type = mt5.ORDER_TYPE_BUY if signal == "bullish" else mt5.ORDER_TYPE_SELL

    curr = df.iloc[-1]
    sl_price = curr['low'] if signal == "bullish" else curr['high']
    sl_dist = abs(price - sl_price)
    tp_dist = 2 * sl_dist
    sl = round(price - sl_dist, 5) if signal == "bullish" else round(price + sl_dist, 5)
    tp = round(price + tp_dist, 5) if signal == "bullish" else round(price - tp_dist, 5)

    if not mt5.symbol_select(symbol, True):
        print(f"!!! >>> Failed to select symbol: {symbol}")
        return

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "price": price,
        "sl": sl,
        "tp": tp,
        "deviation": 10,
        "magic": 234000,
        "comment": f"{signal} engulfing SL/TP",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    result = mt5.order_send(request)

    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Order failed: {result.retcode}")
    else:
        print(f"{signal.capitalize()} trade placed successfully at {price}")
        print(f"SL: {sl} | TP: {tp}")


def run_strategy():
    weekday = datetime.now().weekday()
    # if weekday >= 5:
    #     print("Weekend — skipping execution.")
    #     return

    symbol = "EURUSD"
    connect_mt5()
    df = get_fifteen_min_data(symbol, n=10)
    # print(df.tail(2))
    print(df)

    signal = detect_engulfing(df)
    print(f"Signal: {signal}")

    place_trade(symbol, signal, df)
    shutdown_mt5()

In [6]:
# test the strategy function
run_strategy()

Good! Connected to MT5
                 time     open     high      low    close  tick_volume  \
0 2025-12-26 21:30:00  1.17732  1.17753  1.17724  1.17750          325   
1 2025-12-26 21:45:00  1.17750  1.17766  1.17735  1.17766          400   
2 2025-12-26 22:00:00  1.17765  1.17791  1.17760  1.17764          489   
3 2025-12-26 22:15:00  1.17764  1.17775  1.17751  1.17773          341   
4 2025-12-26 22:30:00  1.17773  1.17816  1.17773  1.17792          473   
5 2025-12-26 22:45:00  1.17792  1.17804  1.17722  1.17748          902   
6 2025-12-26 23:00:00  1.17748  1.17763  1.17720  1.17734          265   
7 2025-12-26 23:15:00  1.17734  1.17740  1.17714  1.17729          268   
8 2025-12-26 23:30:00  1.17729  1.17729  1.17655  1.17687          255   
9 2025-12-26 23:45:00  1.17686  1.17718  1.17675  1.17712         1235   

   spread  real_volume  
0       0            0  
1       0            0  
2       0            0  
3       0            0  
4       0            0  
5       0   

In [7]:
schedule.every(10).seconds.do(run_strategy)

print("------> Strategy scheduler is running every 10 seconds...")

while True:
    schedule.run_pending()
    time.sleep(5)

------> Strategy scheduler is running every 10 seconds...
Good! Connected to MT5
                 time     open     high      low    close  tick_volume  \
0 2025-12-26 21:30:00  1.17732  1.17753  1.17724  1.17750          325   
1 2025-12-26 21:45:00  1.17750  1.17766  1.17735  1.17766          400   
2 2025-12-26 22:00:00  1.17765  1.17791  1.17760  1.17764          489   
3 2025-12-26 22:15:00  1.17764  1.17775  1.17751  1.17773          341   
4 2025-12-26 22:30:00  1.17773  1.17816  1.17773  1.17792          473   
5 2025-12-26 22:45:00  1.17792  1.17804  1.17722  1.17748          902   
6 2025-12-26 23:00:00  1.17748  1.17763  1.17720  1.17734          265   
7 2025-12-26 23:15:00  1.17734  1.17740  1.17714  1.17729          268   
8 2025-12-26 23:30:00  1.17729  1.17729  1.17655  1.17687          255   
9 2025-12-26 23:45:00  1.17686  1.17718  1.17675  1.17712         1235   

   spread  real_volume  
0       0            0  
1       0            0  
2       0            0  
3   

KeyboardInterrupt: 

In [ ]:
import MetaTrader5 as mt5
# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)
 
 
# display data on MetaTrader 5 version
print(mt5.version())
# connect to the trade account without specifying a password and a server
account=17221085
authorized=mt5.login(account)  # the terminal database password is applied if connection data is set to be remembered
if authorized:
    print("connected to account #{}".format(account))
else:
    print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))
 
# now connect to another trading account specifying the password
account=25115284
authorized=mt5.login(account, password="gqrtz0lbdm")
if authorized:
    # display trading account data 'as is'
    print(mt5.account_info())
    # display trading account data in the form of a list
    print("Show account_info()._asdict():")
    account_info_dict = mt5.account_info()._asdict()
    for prop in account_info_dict:
        print("  {}={}".format(prop, account_info_dict[prop]))
else:
    print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))
 
# shut down connection to the MetaTrader 5 terminal
mt5.shutdown()
 